In [76]:
!python -m spacy download pt_core_news_sm
!pip install enelvo
!pip install googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 52.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [77]:
from google.colab import drive
from googletrans import Translator
from enelvo.normaliser import Normaliser
import re
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Pré-processamento

## 0) Base de Dados

In [79]:
norm = Normaliser(tokenizer='readable')
chats_clients = pd.read_csv('/content/drive/Shareddrives/grupo3moshi/chats_clients.csv')
chats_clients.head()

,No,Intencao,Pergunta,Resposta
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [80]:
chats_clients.fillna('erro ao processar a pergunta', inplace=True)

In [81]:
nlp_pt = spacy.load("pt_core_news_sm")

In [82]:
def correct_grammar(text):
  return norm.normalise(text)

## 1) Reconhecimento de Nomes de Entidades

In [83]:
def extract_ents(text):
    # Verify if it is a valid text
    if not isinstance(text, str):
        return []

    #  Use the portuguese model to process the text
    doc_pt = nlp_pt(text)

    # Place named entities in a dictionary
    ents = [{f'{ent.text}': ent.label_} for ent in doc_pt.ents]
    return ents

## 2) Tokenização

A Tokenização consiste no processo de divisão de um texto em partes menores, tokens, sejam elas palavras, caracteres ou até frases, de forma que o modelo de LLM possa processar e manipular os tokens com os algoritmos. Esse processo é a primeira etapa do treinamento do modelo e essencial para o resultado das outras etapas. ([Fonte](https://learn.microsoft.com/pt-br/dotnet/ai/conceptual/understanding-tokens))

In [84]:
def tokenize(text):

  doc = nlp_pt(text)
  tokens = [token.text for token in doc if not token.is_punct]
  return tokens

## 3) Remoção de Stop Words


A etapa de remoção de Stop Words é um processo de refinamento e eliminação de palavras irrelevantes para o processamento do modelo de LLM, ao consultar em uma lista de palavras irrelevantes no idioma em questão. Assim, a dimensionalidade do problema é diminuída e a eficiência do modelo aumentada com a eliminação de palavras irrelevantes, como "a", "e", "de". ([Fonte](https://fineproxy.org/pt/wiki/stopword-removal/#:~:text=A%20remo%C3%A7%C3%A3o%20de%20palavras%20de,automatizado%20e%20classifica%C3%A7%C3%A3o%20de%20textos.))

In [85]:
def remover_stop_words(tokens):
  stop_words = set(stopwords.words('portuguese'))
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
  return filtered_tokens

## 4) Lematização

A Lematização é o processo de extrair o infinitivo dos verbos, de forma que haja uma padronização na utilização das palavras. Isso é importante para a rede reconhecer de forma mais simples os padrões entre os tokens. ([Fonte](https://blog.bitso.com/pt-br/criptomoedas/diferenca-entre-tokenizacao-e-lematizacao#:~:text=Lematiza%C3%A7%C3%A3o%20%C3%A9%20um%20tipo%20de,fim%20de%20simplificar%20a%20palavra.))

In [86]:
def lemmatize(text):
    text = ' '.join(text)
    doc = nlp_pt(text)
    lemmatized_token = [token.lemma_ for token in doc]

    return lemmatized_token

### 5) Bag Of Words

O Bag Of Words cria um vocabulário a partir de um conjunto de documentos e, para cada documento, conta quantas vezes cada palavra do vocabulário aparece, resultando em um vetor de frequências.

In [94]:
def bag_of_words(df, column):
  column_perguntas = [' '.join(sublista) for sublista in df[column]]
  vectorizer = CountVectorizer()
  bag_of_words = vectorizer.fit_transform(column_perguntas)
  return bag_of_words.toarray()

### 6) Remover listas vazias

Essa parte remove as listas vazias para não ter problema ao fazer o bag of words, tentando criar o BoW sendo que não tem palavra alguma.

In [88]:
def remove_empty_lists(df):
    df = df[~df['Pergunta'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
    return df

### 7) Remover números

Remove os números para o bag of words ser mais otimizado. Não há motivo para considerar números no vocabluário.

In [89]:
def remove_numbers(text):
    text_no_numbers = [token for token in text if not token.isdigit()]
    for i, token in enumerate(text_no_numbers):
        for letter in token:
            if letter.isdigit():
                token = token.replace(letter, '')
        text_no_numbers[i] = token
    return text_no_numbers

## 8) Pipeline completa

A fim de simplificar o pré-processamento, junta-se todos os processos empregados na seguite função até então (em desenvolvimento)

In [90]:
def pre_processsing(df):
  df['Pergunta'] = df['Pergunta'].apply(correct_grammar)
  df['Pergunta - NER'] = df['Pergunta'].apply(extract_ents)
  df['Pergunta'] = df['Pergunta'].apply(tokenize)
  df['Pergunta'] = df['Pergunta'].apply(remove_numbers)
  df['Pergunta'] = df['Pergunta'].apply(remover_stop_words)
  df['Pergunta'] = df['Pergunta'].apply(lemmatize)
  df = remove_empty_lists(df)
  df['Resposta'] = df['Resposta'].apply(correct_grammar)
  df['Resposta - NER'] = df['Resposta'].apply(extract_ents)
  df['Resposta'] = df['Resposta'].apply(tokenize)
  df['Resposta'] = df['Resposta'].apply(remover_stop_words)
  df['Resposta'] = df['Resposta'].apply(lemmatize)


### 8.1) Resultado Final

In [91]:
pre_processsing(chats_clients)
chats_clients.head()

ERROR:enelvo.normaliser:Failed to normalise word "paypayginkou"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "zayrucard"!
ERROR:enelvo.normaliser:Failed to normalise word "brastelremit"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "mitsuisumitomo"!
ERROR:enelvo.normaliser:Failed to normalise word "incomodarmuito"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "onegaishimasu"!
ERROR:enelvo.normaliser:Failed to normalise word "ゆうちょ銀行"!
ERROR:enelvo.normaliser:Failed to normalise word "horariohttps"!
ERROR:enelvo.normaliser:Failed to normalise word "brastelremit"!
ERROR:enelvo.normaliser:Failed to normalise word "horariohttps"!
ERROR:enelvo.normaliser:Failed to normalise word "brastelremit"!
ERROR:enelvo.normaliser:F

,No,Intencao,Pergunta,Resposta,Pergunta - NER
0,1,Como depositar,"[bom, dia, tudo, bem, gostar, saber, sobre, ca...","Bom dia! Sim, o sr pode utilizar o cartão de d...",[{'marta verde': 'LOC'}]
1,2,Como fazer remessa,"[enviar, dinheiro, Japão]","Para se inscrever no serviço de remessa, por f...",[{'do japão': 'LOC'}]
2,3,Tempo de remessa,"[quanto, tempo, levar, beneficiário, receber, ...","Via de regra, as remessas serão pagas via PIX ...",[]
3,4,"Pedido de envio via metodo ""ByPhone""","[bom, tarde, acabar, fazer, transferência, +, ...",iremos processar a sua solicitacao.\nMuito obr...,[]
4,5,"Pedido de envio via metodo ""ByPhone""","[poder, fazer, remesso, yenes, bbb, rer, obrig...",iremos processar a sua solicitacao.\nMuito obr...,[{'bbb': 'MISC'}]


In [95]:
bow_pergunta = bag_of_words(chats_clients, 'Pergunta')

print(bow_pergunta)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
